## Agile Hardware Design
***
# Testing

<img src="../images/logo.svg" alt="agile hardware design logo" style="float:right"/>

###  Modified by Peter Hanping Chen based on 
### Prof. Scott Beamer (sbeamer@ucsc.edu) 
### [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)
### and 
### UC Berkeley Bootcamp Scala configuration file (load_ivy.sc)
### https://github.com/freechipsproject/chisel-bootcamp


## Plan for Today

* Testing overview
* Testing a combinational unit
* Tidying up with ScalaTest
* Testing a Decoupled (stateful) unit

## Loading The Chisel Library Into a Notebook

In [1]:
//interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
//val path = System.getProperty("user.dir") + "/source/chisel_deps.sc"
println("path: "+path)
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/009_Test/001_Code/source/load-ivy.sc


path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/009_Test/001_Code/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test
import org.scalatest.flatspec.AnyFlatSpec

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

import org.scalatest.flatspec.AnyFlatSpec

In [3]:
class ConvUIntToOH(inWidth: Int) extends Module {
    val outWidth = 1 << inWidth
    val io = IO(new Bundle {
        val in  = Input(UInt(inWidth.W))
        val out = Output(UInt(outWidth.W))
    })
    require (inWidth > 0)
    def helper(index: Int): UInt = {
        if (index < outWidth-1) Cat(helper(index+1), io.in === index.U)
        else io.in === index.U
    }
    io.out := helper(0)
//     io.out := UIntToOH(io.in)  // Standard library implementation
    printf("%d -> %b\n", io.in, io.out)
}

defined class ConvUIntToOH

In [4]:
//printVerilog(new ConvUIntToOH(2))
println (getVerilog(new ConvUIntToOH(2)))

Elaborating design...
Done elaborating.
module ConvUIntToOH(
  input        clock,
  input        reset,
  input  [1:0] io_in,
  output [3:0] io_out
);
  wire  _T = io_in == 2'h3; // @[cmd2.sc 10:20]
  wire  _T_1 = io_in == 2'h2; // @[cmd2.sc 9:60]
  wire  _T_3 = io_in == 2'h1; // @[cmd2.sc 9:60]
  wire [2:0] _T_4 = {_T,_T_1,_T_3}; // @[Cat.scala 30:58]
  wire  _T_5 = io_in == 2'h0; // @[cmd2.sc 9:60]
  assign io_out = {_T_4,_T_5}; // @[Cat.scala 30:58]
  always @(posedge clock) begin
    `ifndef SYNTHESIS
    `ifdef PRINTF_COND
      if (`PRINTF_COND) begin
    `endif
        if (~reset) begin
          $fwrite(32'h80000002,"%d -> %b\n",io_in,io_out); // @[cmd2.sc 14:11]
        end
    `ifdef PRINTF_COND
      end
    `endif
    `endif // SYNTHESIS
  end
endmodule



## Why Test?

* Who wants your hardware if it doesn't work?
* How do you prove to yourself it works?
* How do you prove to others it works?
* How do you even develop it?

## Goals of Today's Testing Lecture

- Develop techniques/abstractions to improve testing productivity
- View testing as an _integral_ part of development (not just verification)
- Learn more sophisticated ways to test in Chisel

## 3 Main Components of Testing
#### 1) How do you generate test cases?
- Human-generated - best for simple cases or edge cases
- Synthetically generated - exhaustive or (directed) random
#### 2) How do you know what is the correct response to test?
- Human-generated - brittle and best to avoid after initial bootstrap
- Model-generated - highly preferable, but need to also test model
#### 3) How do you simulate/execute/script test?
- In general - consider need for flexibility, portability, speed
- Today: simulate with treadle, execute with ChiselTest, organize with ScalaTest

## Considerations for Designing Tests

#### 1) What do you need to get started?
- Close the loop early in order to do _test-driven development (TDD)_

#### 2) What is _coverage_ needed for this problem?
- Consider what you will need to test, and how to be sure you covered it

#### 3) Should you treat design under test as _opaque_ or _clear_?
- Both!
- Users see _opaque_ module, so should properly implement specified interface
- Knowing implementation (_clear_) can help focus tests on likely edge cases

## When to Use Testing

#### 1) Helpful in many places and not just final verification
- Initial development
- Continuous integration running in background
- Working with others (i.e. checking external contributions)
- Design space exploration

#### 2) Consider testing early in process and design for it
- Consider design abstractions and module boundaries to ease testing
- Combinational modules can be easier to test, so place state elements deliberately

## Testing in Chisel

#### 1) Are generators harder to test?
- Yes, but can parameterize test generation too!
- Can amortize test development over all instances (produced by generator)

#### 2) Chisel Test: https://github.com/ucb-bar/chiseltest
- Can write testbenches directly in Scala
- Runs as a Scala program that communicates with simulation of design
- Upcoming Chisel library for testing (we have already been using it)

#### 3) Simulation options
##### 3.1 Tread: https://github.com/chipsalliance/treadle 
    - default FIRRTL simulator, implemented directly in Scala
    - Default, easy to get going, and fastest for small designs
##### 3.2 Verilator: https://www.veripool.org/wiki/verilator
    - fast open-source Verilog simulation, can talk to ChiselTest 
    - Inter-process communication and ChiselTest can slow down overall
##### 3.3 Other simulators
    - Can simulate Verilog from Chisel, but won't be able to talk back to ChiselTest

## Testing a Combinational Component

#### 1) Stateless (combinational) modules are easier to test since each test/cycle is independent

#### 2) Consider:
  - range of possible inputs
  - range of generator parameters
  - parameters' impact on input space

#### 3) If input space is sufficiently small, may be able to _exhaustively_ test
  - May be able to make sufficiently small by constraining parameters

## 9.11 Example: Test Combonational Component

- Sign & Magnitude: Comment out the Sign
- Add Module Implementation

In [64]:
// We commenet out sign
class SignMagAdd(val w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(w.W))
        val out = Output(UInt(w.W))
    })
    io.out := io.in
}

defined class SignMagAdd

In [48]:
// Comment out the sign
/*
class SignMag(w: Int) extends Bundle {
    val sign = Bool()
    val magn = UInt(w.W)
}

class SignMagAdd(val w: Int) extends Module {
    val io = IO(new Bundle {
        val in0 = Input(new SignMag(w))
        val in1 = Input(new SignMag(w))
        val out = Output(new SignMag(w))
    })
    when (io.in0.sign === io.in1.sign) {
        io.out.sign := io.in0.sign
        io.out.magn := io.in0.magn + io.in1.magn    
    } .elsewhen (io.in0.magn > io.in1.magn) {
        io.out.sign := io.in0.sign
        io.out.magn := io.in0.magn - io.in1.magn     
    } .otherwise {
        io.out.sign := io.in1.sign
        io.out.magn := io.in1.magn - io.in0.magn   
    }
}
*/

In [52]:
// print Hardware Verilog
println (getVerilog(new SignMagAdd(2)))

Elaborating design...
Done elaborating.
module SignMagAdd(
  input        clock,
  input        reset,
  input  [1:0] io_in,
  output [1:0] io_out
);
  assign io_out = io_in; // @[cmd48.sc 6:12]
endmodule



In [53]:
test(new SignMagAdd(4)) { c =>
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)
}

Elaborating design...
Done elaborating.
test SignMagAdd Success: 0 tests passed in 2 cycles in 0.001311 seconds 1525.47 Hz


In [51]:
// COmment out the sign
/* 
test(new SignMagAdd(4)) { c =>
    c.io.in0.sign.poke(false.B)
    c.io.in0.magn.poke(1.U)

    c.io.in1.sign.poke(false.B)
    c.io.in1.magn.poke(2.U)

    c.io.out.sign.expect(false.B)
    c.io.out.magn.expect(3.U)
}
*/

## 9.12 Example: modelAdd
- Make a Mode
- Use Scala to generate the desired behavior
  * Can simply produce right output or even make a class
- Be sure to model truncating/wrapping effects of data widths

In [65]:
// Scala Test
def modelAdd(a: Int, b: Int, w: Int): Int = {
    require(w > 0)
    require(w < 32)
    val mask = (1 << w) - 1
    val sum = a + b
    if (sum < 0) -((-sum) & mask)
    else sum & mask
}

println ("modelAdd(4, 4, 4): " + modelAdd(4, 4, 4))
println ("modelAdd(2, 3, 4): " + modelAdd(2, 3, 4))
println ("modelAdd(-1, 5, 4): " + modelAdd(-1, 5, 4))
println ("modelAdd(1, -1, 4): " + modelAdd(1, -1, 4))

modelAdd(4, 4, 4): 8
modelAdd(2, 3, 4): 5
modelAdd(-1, 5, 4): 4
modelAdd(1, -1, 4): 0


defined function modelAdd

In [36]:
// getverilog only pass the class. It cannot pass method.
//println (getVerilog(new modelAdd(4,4,4)))

## 9.13 Example: PassThroughTest

In [66]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.experimental.BundleLiterals._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.experimental._

import chisel3.experimental.BundleLiterals._

import chisel3.tester._

import chisel3.tester.RawTester.test

In [69]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

defined class PassthroughGenerator

In [68]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

Elaborating design...
Done elaborating.
test PassthroughGenerator Success: 0 tests passed in 2 cycles in 0.001510 seconds 1324.72 Hz


In [70]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.clock.step(1)    // advance the clock
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.clock.step(1)    // advance the clock
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.clock.step(1)    // advance the clock
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

Elaborating design...
Done elaborating.
test PassthroughGenerator Success: 0 tests passed in 5 cycles in 0.002657 seconds 1882.12 Hz


In [46]:
/*
def testAdd(a: Int, b: Int, c: SignMagAdd, verbose: Boolean=true) {
    c.io.in0.sign.poke((a<0).B)
    c.io.in0.magn.poke(math.abs(a).U)
    c.io.in1.sign.poke((b<0).B)
    c.io.in1.magn.poke(math.abs(b).U)
    val outSignStr = if (c.io.out.sign.peek().litToBoolean) "-" else ""
    val outMag = c.io.out.magn.peek().litValue
    if (verbose)
        println(s"  in: $a + $b  out: $outSignStr$outMag")
    if (modelAdd(a,b,c.w) != 0)
        c.io.out.sign.expect((modelAdd(a,b,c.w) < 0).B)  // what is buggy here?
    c.io.out.magn.expect(math.abs(modelAdd(a,b,c.w)).U)
}

test(new SignMagAdd(4)) { c =>
    testAdd(2,3,c)
    testAdd(-1,5,c)
    testAdd(1,-1,c)
}
*/

## Combo. Example - Sign & Magnitude Add - Test Exhaustively

In [6]:
/*
def testAll(w: Int) {
    val maxVal = (1<<w) - 1
    test(new SignMagAdd(w)) { dut =>
        for (a <- -maxVal to maxVal) {
            for (b <- -maxVal to maxVal) {
                testAdd(a,b,dut,false)
            }
        }
    }
}

testAll(2)
*/

## Combo. Example - Sign & Magnitude Add - Random Test

In [6]:
/*
def testRandomAdd(dut: SignMagAdd) {
    def genInput() = {
        val limit = 1 << dut.w
        val magn = scala.util.Random.nextInt(limit)
        val neg = scala.util.Random.nextBoolean
        if (neg) -magn else magn
    }
    testAdd(genInput(), genInput(), dut)
}

def testRandomly(w: Int, numTrials: Int) {
    test(new SignMagAdd(w)) { dut =>
        for (t <- 0 until numTrials)
            testRandomAdd(dut)
    }
}

testRandomly(4, 5)
*/

## ScalaTest

* Helpful [library](https://www.scalatest.org) to organize and group tests

* `sbt` is aware of it
  * Running `test` automatically runs all ScalaTests it can find
  * Can also use `testOnly package.class` to only test `package.class`

* ChiselTest can interoperate with it, and we have been using it in the homework already

## Combo. Example - Sign & Magnitude Add - with ScalaTest

In [6]:
/*
class SignMagAddTest(w: Int) extends AnyFlatSpec with ChiselScalatestTester {
    behavior of s"SignMagAdd($w)"
    it should "1 + 2 = 3" in {
        test(new SignMagAdd(w)) { dut =>
            testAdd(1,2,dut)
        }
    }
    it should "1 - 1 = 0" in {
        test(new SignMagAdd(w)) { dut =>
            testAdd(1,-1,dut)
        }
    }
}

(new SignMagAddTest(4)).execute()
*/

## Combo. Example - Sign & Magnitude Add - Bundle Literals

* Experimental [feature](https://www.chisel-lang.org/chisel3/docs/appendix/experimental-features#bundle-literals) to specify a Bundle all at once

In [6]:
/*
import chisel3.experimental.BundleLiterals._

test(new SignMagAdd(4)) { c =>
    val b0 = chiselTypeOf(c.io.in0).Lit(_.sign->false.B, _.magn->2.U)
    val b1 = (new SignMag(4)).Lit(_.sign->false.B, _.magn->2.U)
    val s  = chiselTypeOf(c.io.out).Lit(_.sign->false.B, _.magn->4.U)
    c.io.in0.poke(b0)
    c.io.in1.poke(b1)
    c.io.out.expect(s)
}
*/

## 9.14 Seq. Example - Queue - Intro

* Testing stateful things is more difficult because prior history (in test) matters
  * Causes large state space explosion
  * Exhaustive testing is unlikely to be feasible
* _Today:_ let's test out Chisel's `Queue` (stateful and uses `Decoupled`)

```scala
    Queue(UInt(n.W), numEntries, pipe=true, flow=false)
```

## Seq. Example - Queue - Model Implementation

* Be careful when modeling interactions with registers
  * Don't want register input to be available at register output too soon
* _Easy fix (for most of the time):_ ensure registers are read first in a cycle before written
* _Alternative:_ buffer register inputs and apply them all at once when cycle advances

In [7]:
class QueueModel(numEntries: Int) {
    val mq = scala.collection.mutable.Queue[Int]()

    var deqReady = false    // set externally
    def deqValid() = mq.nonEmpty
    // be sure to call attemptDeq before attemptEnq within a cycle
    def attemptDeq() = if (deqReady && deqValid) Some(mq.dequeue()) else None

    def enqReady() = mq.size < numEntries || (mq.size == numEntries && deqReady)    // pipe = true
    def attemptEnq(elem: Int): Unit = if (enqReady()) mq += elem    // implies enqValid
}

defined class QueueModel

## Seq. Example - Queue - Model Demo Small

In [8]:
val qm = new QueueModel(2)

// attempt push 1
qm.enqReady()
qm.deqValid()
qm.deqReady = false
qm.attemptDeq()
qm.attemptEnq(1)

// attempt push 2 & pop 1
qm.enqReady()
qm.deqValid()
qm.deqReady = true
qm.attemptDeq()
qm.attemptEnq(2)

qm: QueueModel = ammonite.$sess.cmd6$Helper$QueueModel@6cf5d3b6
res7_1: Boolean = true
res7_2: Boolean = false
res7_4: Option[Int] = None
res7_6: Boolean = true
res7_7: Boolean = true
res7_9: Option[Int] = Some(1)

## Seq. Example - Queue - Model Demo Long

In [9]:
val qm = new QueueModel(2)

for (i <- 1 to 6) {
    qm.deqReady = i > 3
    print(s"deqV: ${qm.deqValid()}\tdeqR: ${qm.deqReady}\tdeqB: ${qm.attemptDeq()}")
    println(s"\tenqV: true\tenqR: ${qm.enqReady()}\tenqB: $i")
    qm.attemptEnq(i)
}

deqV: false	deqR: false	deqB: None	enqV: true	enqR: true	enqB: 1
deqV: true	deqR: false	deqB: None	enqV: true	enqR: true	enqB: 2
deqV: true	deqR: false	deqB: None	enqV: true	enqR: false	enqB: 3
deqV: true	deqR: true	deqB: Some(1)	enqV: true	enqR: true	enqB: 4
deqV: true	deqR: true	deqB: Some(2)	enqV: true	enqR: true	enqB: 5
deqV: true	deqR: true	deqB: Some(4)	enqV: true	enqR: true	enqB: 6


qm: QueueModel = ammonite.$sess.cmd6$Helper$QueueModel@60ce44bc

## Seq. Example - Queue - Manually Comparing to Model

In [10]:
test(new Queue(UInt(32.W), 2, pipe=true, flow=false)) { dut =>
    val qm = new QueueModel(2)
    // always dequeue (for this example)
    qm.deqReady = true
    dut.io.deq.ready.poke(qm.deqReady.B)

    // try to dequeue on empty
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult0 = qm.attemptDeq()
    if (deqResult0.isDefined) dut.io.deq.bits.expect(deqResult0.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(false.B)
    dut.io.enq.bits.poke(0.U)
    dut.clock.step()
    
    // enqueue 1
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult1 = qm.attemptDeq()
    if (deqResult1.isDefined) dut.io.deq.bits.expect(deqResult1.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(true.B)
    dut.io.enq.bits.poke(1.U)
    qm.attemptEnq(1)
    dut.clock.step()
    
    // enqueue nothing, dequeue 1
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult2 = qm.attemptDeq()
    if (deqResult2.isDefined) dut.io.deq.bits.expect(deqResult2.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(false.B)
    dut.io.enq.bits.poke(0.U)
    dut.clock.step()
}

Elaborating design...
Done elaborating.
test Queue Success: 0 tests passed in 5 cycles in 0.041302 seconds 121.06 Hz


## Seq. Example - Queue - Automate Interaction

In [11]:
def simCycle(dut: Queue[UInt], qm: QueueModel, enqValid: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    dut.io.deq.ready.poke(qm.deqReady.B)
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult = qm.attemptDeq()
    if (deqResult.isDefined)
        dut.io.deq.bits.expect(deqResult.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(enqValid.B)
    dut.io.enq.bits.poke(enqData.U)
    if (enqValid)
        qm.attemptEnq(enqData)
    dut.clock.step()
    println(qm.mq)
}

test(new Queue(UInt(32.W), 2, pipe=true, flow=false)) { dut =>
    val qm = new QueueModel(2)
    simCycle(dut, qm, false, false)
    simCycle(dut, qm, true, false, 1)
    simCycle(dut, qm, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue()
test Queue Success: 0 tests passed in 5 cycles in 0.008584 seconds 582.49 Hz


defined function simCycle

## Seq. Example - Queue - Test Fill & Drain

In [12]:
def testFillAndDrain(numEntries: Int, w: Int) {
    test(new Queue(UInt(w.W), numEntries, pipe=true, flow=false)) { dut =>
        val qm = new QueueModel(numEntries)
        for (x <- 0 to numEntries) {  // fill
            simCycle(dut, qm, true, false, x)
        }
        for (x <- 0 to numEntries) {  // drain
            simCycle(dut, qm, false, true)
        }
    }
}

testFillAndDrain(3, 32)

Elaborating design...
Done elaborating.
Queue(0)
Queue(0, 1)
Queue(0, 1, 2)
Queue(0, 1, 2)
Queue(1, 2)
Queue(2)
Queue()
Queue()
test Queue Success: 0 tests passed in 10 cycles in 0.023066 seconds 433.54 Hz


defined function testFillAndDrain

## Seq. Example - Queue - Test Randomly

In [13]:
def testRandomly(numEntries: Int, w: Int, numTrials: Int) {
    test(new Queue(UInt(w.W), numEntries, pipe=true, flow=false)) { dut =>
        val qm = new QueueModel(numEntries)
        for (i <- 1 until numTrials) {
            val tryEnq = scala.util.Random.nextBoolean
            val tryDeq = scala.util.Random.nextBoolean
            simCycle(dut, qm, tryEnq, tryDeq, i)
        }
    }
}

testRandomly(2, 32, 5)

Elaborating design...
Done elaborating.
Queue()
Queue(2)
Queue(2, 3)
Queue(2, 3)
test Queue Success: 0 tests passed in 6 cycles in 0.009316 seconds 644.05 Hz


defined function testRandomly

## Testing Advice

### Get humans out of the loop
  * Humans should help make tests, but not perform them
  * Print statements & waveforms are for debugging but not testing

### Random may not yield great coverage
  * In large test space, may have low probability of reaching interesting corner case
  * May want to _seed_ to get reproducibility

### Assertions are helpful, but do not replace need for testing
  * Tests don't just ensure consistent state, but also provide test stimuli
  * Assertions best for catching issues early (in simulation time) before they silently cause problems later
    * Consider adding them after debugging subtle bug